In [ ]:
# STROTSS Style Transfer Notebook

In [ ]:
#### [Style Transfer by Relaxed Optimal Transport and Self-Similarity (STROTSS)](https://arxiv.org/abs/1904.12785)
Code from: https://github.com/futscdav/strotss
Notebook by: Peter Schaldenbrand

## Download the strotss code from GitHub

In [ ]:
import os
if not os.path.exists('/content/strotss'):
    !git clone https://github.com/futscdav/strotss.git
os.chdir('/content/strotss')
from strotss import *

## Helper Functions

In [ ]:
import torch
import requests
import PIL.Image
from io import BytesIO
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if not torch.cuda.is_available():
    print('YOU ARE NOT USING A GPU.  IT\'S GONNA BE REAAALLLLY SLOW')
    print('Go to the top of the page.  Click Runtime -> Change Runtime Type -> Hardware accelerator')
    print('From the dropdown, select GPU and rerun all this stuff')

def pil_loader_internet(url):
    response = requests.get(url)
    img = PIL.Image.open(BytesIO(response.content))
    return img.convert('RGB')

def show_img(img):
    # Code for displaying at actual resolution from:
    # https://stackoverflow.com/questions/28816046/displaying-different-images-with-actual-size-in-matplotlib-subplot
    dpi = 80
    height, width, depth = img.shape
    figsize = width / float(dpi), height / float(dpi)
    plt.figure(figsize=figsize)

    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.show()


def plot_style_and_content(style, content):
    fig, ax = plt.subplots(1,2, figsize=(10,5))
    ax[0].imshow(content)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].set_title('Content')
    ax[1].imshow(style)
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    ax[1].set_title('Style')
    plt.show()

## Define STROTSS function

In [ ]:
# Redefine the STROTSS function to put some debugging statements in

def strotss(content_pil, style_pil, content_weight=1.0*16.0, device='cuda:0', space='uniform'):
    content_np = pil_to_np(content_pil)
    style_np = pil_to_np(style_pil)
    content_full = np_to_tensor(content_np, space).to(device)
    style_full = np_to_tensor(style_np, space).to(device)

    lr = 2e-3
    extractor = Vgg16_Extractor(space=space).to(device)

    scale_last = max(content_full.shape[2], content_full.shape[3])
    scales = []
    for scale in range(10):
        divisor = 2**scale
        if min(content_pil.width, content_pil.height) // divisor >= 33:
            scales.insert(0, divisor)
    
    clow = -1.0 if space == 'uniform' else -1.7
    chigh = 1.0 if space == 'uniform' else 1.7

    for scale in scales:
        # rescale content to current scale
        content = tensor_resample(content_full, [ content_full.shape[2] // scale, content_full.shape[3] // scale ])
        style = tensor_resample(style_full, [ style_full.shape[2] // scale, style_full.shape[3] // scale ])
        print(f'Optimizing at resoluton [{content.shape[2]}, {content.shape[3]}]')

        # upsample or initialize the result
        if scale == scales[0]:
            # first
            result = laplacian(content) + style.mean(2,keepdim=True).mean(3,keepdim=True)
        elif scale == scales[-1]:
            # last 
            result = tensor_resample(result, [content.shape[2], content.shape[3]])
            lr = 1e-3
        else:
            result = tensor_resample(result, [content.shape[2], content.shape[3]]) + laplacian(content)

        # do the optimization on this scale
        result = optimize(result, content, style, scale, content_weight=content_weight, lr=lr, extractor=extractor)

        # Show intermediate result
        result_image = tensor_to_np(torch.clamp(result, clow, chigh)) # 
        # renormalize image
        result_image -= result_image.min()
        result_image /= result_image.max()
        show_img(result_image)

        # next scale lower weight
        content_weight /= 2.0

    result_image = tensor_to_np(tensor_resample(torch.clamp(result, clow, chigh), [content_full.shape[2], content_full.shape[3]])) # 
    # renormalize image
    result_image -= result_image.min()
    result_image /= result_image.max()
    return np_to_pil(result_image * 255.)

#@title Run STROTSS
#@markdown Put publicly accessible URLs to images for content_url and style_url
content_url = 'https://mymodernmet.com/wp/wp-content/uploads/2021/08/frog-photos-ajar-setiadi-4.jpeg'  #@param {type:"string"}
style_url = 'https://m.media-amazon.com/images/I/91iS91eizUL._AC_SL1500_.jpg'  #@param {type:"string"}

content_pil = pil_loader_internet(content_url)
style_pil = pil_loader_internet(style_url)
plot_style_and_content(style_pil, content_pil)

max_width = 512  #@param {type:"integer"}

#@markdown How much weight to give to content. 
content_weight = 0.5 #@param {type:"number"}
content_weight *= 16.0 

result = strotss(pil_resize_long_edge_to(content_pil, max_width), 
        pil_resize_long_edge_to(style_pil, max_width), 
        content_weight, device, "vgg")
print('Final Result')
show_img(pil_to_np(result))